**Aayusha Singh**

  **2330817**
  

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

**Implementing Softmax Function**

In [ ]:
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Preventing overflow by subtracting max value
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)  # Normalizing to sum up to 1

**Prediction Function**

In [ ]:
def predict_softmax(X, W, b):
    logits = np.dot(X, W) + b  # Computing raw scores
    probabilities = softmax(logits)  # Converting the scores to probabilities
    return np.argmax(probabilities, axis=1)  # Returning the class with highest probability

**Loss and Cost Functions**

In [ ]:
def loss_softmax(y_pred, y):
    return -np.sum(y * np.log(y_pred + 1e-9)) / y.shape[0]  # Cross-entropy loss

# Cost function calculates the average loss for all data points.
def cost_softmax(X, y, W, b):
    logits = np.dot(X, W) + b
    y_pred = softmax(logits)
    return loss_softmax(y_pred, y)

**Computing Gradients**

In [ ]:
def compute_gradient_softmax(X, y, W, b):
    logits = np.dot(X, W) + b
    y_pred = softmax(logits)
    grad_W = np.dot(X.T, (y_pred - y)) / X.shape[0]  # Gradient for weights
    grad_b = np.sum(y_pred - y, axis=0) / X.shape[0]  # Gradient for biases
    return grad_W, grad_b

**Gradient Descent Algorithm**

In [ ]:
def gradient_descent_softmax(X, y, W, b, alpha, n_iter, show_cost=False):
    cost_history = []
    for i in range(n_iter):
        grad_W, grad_b = compute_gradient_softmax(X, y, W, b)
        W -= alpha * grad_W  # Updating weights
        b -= alpha * grad_b  # Updating biases
        cost = cost_softmax(X, y, W, b)  # Computing cost
        cost_history.append(cost)
        if show_cost and i % 100 == 0:
            print(f"Iteration {i}, Cost: {cost:.4f}")  # Printing the cost every 100 iterations
    return W, b, cost_history

**Q1: Is extracting pixel values sufficient for effective feature extraction?**

No, because raw pixels lack structural information. Feature extraction improves classification accuracy.

**Loading and Preparing the MNIST Dataset**

In [ ]:
def load_and_prepare_mnist(csv_file, test_size=0.2, random_state=42):
    df = pd.read_csv(csv_file)  # Reading the dataset
    y = df.iloc[:, 0].values  # First column contains labels
    X = df.iloc[:, 1:].values / 255.0  # Normalizing the pixel values (0-255 → 0-1)
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

**Evaluating the Model**

In [ ]:
def evaluate_classification(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    return cm, precision, recall, f1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Loading the MNIST dataset**

In [ ]:
X_train, X_test, y_train, y_test = load_and_prepare_mnist('/content/drive/MyDrive/Colab Notebooks/mnist_dataset.csv')

**Converting the numerical labels into binary vectors.**

In [ ]:
encoder = OneHotEncoder(sparse_output=False)
y_train = encoder.fit_transform(y_train.reshape(-1, 1))
y_test = encoder.transform(y_test.reshape(-1, 1))

**Model Initialization**

**Initializing the weights and biases with small random values.**

In [ ]:
num_features = X_train.shape[1]  # Number of input features (pixels)
num_classes = y_train.shape[1]  # Number of classes (digits 0-9)
W = np.random.randn(num_features, num_classes) * 0.01  # Small random weights
b = np.zeros(num_classes)  # Bias initializing to zeros

**Training the Model**

In [ ]:
alpha = 0.1
n_iter = 1000
W_opt, b_opt, cost_history = gradient_descent_softmax(X_train, y_train, W, b, alpha, n_iter, show_cost=True)


Iteration 0, Cost: 2.1935
Iteration 100, Cost: 0.6066
Iteration 200, Cost: 0.4894
Iteration 300, Cost: 0.4409
Iteration 400, Cost: 0.4128
Iteration 500, Cost: 0.3940
Iteration 600, Cost: 0.3802
Iteration 700, Cost: 0.3695
Iteration 800, Cost: 0.3609
Iteration 900, Cost: 0.3537


**Plotting Cost Function Convergence**

**Visualizing how the cost function decreases over time.**

In [ ]:
plt.plot(cost_history)
plt.title('Cost Function vs. Iterations')
plt.xlabel('Iterations')
plt.ylabel('Cost')
plt.grid(True)
plt.show()

 **Model Evaluation**

 **Using the trained model to predict on the test set.**

In [ ]:
# Part 12: Model Evaluation
# Using the trained model to predict on the test set.
y_pred_test = predict_softmax(X_test, W_opt, b_opt)
y_test_labels = np.argmax(y_test, axis=1)  # Converting the labels back to numbers
cm, precision, recall, f1 = evaluate_classification(y_test_labels, y_pred_test)

**Printing the evaluation metrics**

In [ ]:

print("\nConfusion Matrix:")
print(cm)
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")



Confusion Matrix:
[[1126    0    5    2    3   12    9    2   13    3]
 [   0 1274    7   11    1    5    1    4   18    1]
 [   1   16 1029   16   19    4   27   25   31    6]
 [   8    5   33 1052    1   53    8    8   30   21]
 [   1    5    7    1 1094    0   10    4    4   50]
 [  22   14   13   43   11  925   14    7   41   14]
 [   7    2   10    1   11   15 1119    2   10    0]
 [   7   26   24    5   16    2    0 1183    7   29]
 [   9   27   15   37    9   30   13    6  999   15]
 [   8    6   10   18   39    9    0   39   10 1055]]
Precision: 0.90
Recall: 0.90
F1-Score: 0.90


**Visualizing Confusion Matrix**

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
cax = ax.imshow(cm, cmap='copper_r', interpolation='nearest')
ax.set_xticks(np.arange(num_classes))
ax.set_yticks(np.arange(num_classes))
ax.set_xticklabels([str(i) for i in range(num_classes)], fontsize=10)
ax.set_yticklabels([str(i) for i in range(num_classes)], fontsize=10)
plt.colorbar(cax)
plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
plt.xlabel('Predicted Label', fontsize=12, fontweight='bold')
plt.ylabel('Actual Label', fontsize=12, fontweight='bold')

# Adding the numbers to each cell in the confusion matrix
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, cm[i, j], ha='center', va='center', color='yellow' if cm[i, j] > cm.max()/2 else 'red', fontsize=10)

plt.show()

**Q2: Provide an interpretation of the output.**

Logistic regression works for linearly separable data but fails for non-linearly separable patterns, needing advanced models like neural networks.

**Q3: Challenges faced during implementation?**

Handling numerical stability, ensuring correct one-hot encoding, tuning learning rates, and preventing overfitting were key challenges.